<a href="https://colab.research.google.com/github/sks9699/Api/blob/master/leaf_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob
import matplotlib.pyplot as plt


2024-02-21 06:35:32.898141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 06:35:32.898241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 06:35:33.159017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import splitfolders
IMAGE_SIZE = [224, 224]

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio("/kaggle/input/species-of-leaf/leaf",
                   output="./output",
                   seed=42,
                   ratio=(.7, .2, .1),
                   group_prefix=None,
                   move=False
                  )


Copying files: 0 files [00:00, ? files/s]
Copying files: 1 files [00:00,  3.51 files/s]
Copying files: 5 files [00:00, 14.90 files/s]
Copying files: 8 files [00:00, 19.08 files/s]
Copying files: 11 files [00:00, 21.65 files/s]
Copying files: 14 files [00:00, 23.54 files/s]
Copying files: 18 files [00:00, 26.45 files/s]
Copying files: 22 files [00:00, 28.89 files/s]
Copying files: 26 files [00:01, 30.38 files/s]
Copying files: 30 files [00:01, 29.84 files/s]
Copying files: 34 files [00:01, 29.85 files/s]
Copying files: 38 files [00:01, 28.57 files/s]
Copying files: 42 files [00:01, 29.43 files/s]
Copying files: 46 files [00:01, 30.44 files/s]
Copying files: 50 files [00:01, 32.20 files/s]
Copying files: 54 files [00:01, 31.98 files/s]
Copying files: 58 files [00:02, 33.88 files/s]
Copying files: 62 files [00:02, 34.42 files/s]
Copying files: 66 files [00:02, 34.02 files/s]
Copying files: 70 files [00:02, 33.90 files/s]
Copying files: 74 files [00:02, 34.87 files/s]
Copying files: 78 fi

In [ ]:
train_directory ='/kaggle/working/output/train'
test_directory ='/kaggle/working/output/test'
val_directory ='/kaggle/working/output/val'

In [ ]:
# import shutil

# # Specify the directory you want to delete
# directory_to_delete = "/kaggle/working/output/train/leaf"

# # Use shutil.rmtree() to recursively remove the directory and its contents
# shutil.rmtree(directory_to_delete)

# print("Directory deleted successfully.")


Directory deleted successfully.


In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights ='imagenet',include_top = False)
for layer in vgg.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
folders = glob('/kaggle/working/output/train')
len(folders)

1

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']  # Change 'metrices' to 'metrics'
)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range =0.2,
                                  zoom_range =0.2,
                                  horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(train_directory,
                                                 target_size =(224,224),
                                                 batch_size = 32,
                                                 class_mode ='categorical'
                                                )

test_set = test_datagen.flow_from_directory(test_directory,
                                                 target_size =(224,224),
                                                 batch_size = 32,
                                                 class_mode ='categorical'
                                                )

Found 3146 images belonging to 12 classes.
Found 462 images belonging to 12 classes.


In [ ]:
print(len(training_set))
print(len(test_set))

99
15


In [ ]:
r = model.fit_generator(
    training_set,
#     steps_per_epochs=16,
    validation_data=test_set,
    epochs =5,
    steps_per_epoch=16,
    validation_steps=len(test_set)
)

/tmp/ipykernel_34/1341036650.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node AssignAddVariableOp defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 534, in dispatch

In [ ]:
plt.plot(r,history['loss'], label='train loss')
plt.plot(r,history['val_loss'], label='val loss')
plt.legend()
plt.show()